<a href="https://colab.research.google.com/github/SPORTNERHSE/SPORTNER/blob/master/NER_regex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pymystem3==0.1.10
!pip install pymorphy2[fast]
!pip install spacy
!pip install nltk

  Found existing installation: pymystem3 0.2.0
    Uninstalling pymystem3-0.2.0:
      Successfully uninstalled pymystem3-0.2.0
     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 7.1MB 8.1MB/s 
     |████████████████████████████████| 378kB 44.4MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp36-cp36m-linux_x86_64.whl size=843813 sha256=23e03b3e7727601cd7299bf5fc8e5bf95d59cbcdedd4c513af2c3562b43d5e95
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG


In [0]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.tokenize import PunktSentenceTokenizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
import string
from gensim.utils import tokenize
from gensim.summarization.textcleaner import split_sentences
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from nltk.corpus import stopwords
from string import punctuation
import re, os, json
import regex
import spacy
import nltk
nlp = spacy.load("en_core_web_sm")
mystem = Mystem()
morph = MorphAnalyzer()

In [0]:
import pandas as pd
import re

In [0]:
df = pd.read_csv('result.tar.gz', sep='\t', error_bad_lines=False)

b'Skipping line 37039: expected 12 fields, saw 15\nSkipping line 45611: expected 12 fields, saw 15\n'
b'Skipping line 354339: expected 12 fields, saw 18\nSkipping line 360804: expected 12 fields, saw 102\nSkipping line 360819: expected 12 fields, saw 78\nSkipping line 379126: expected 12 fields, saw 14\nSkipping line 444446: expected 12 fields, saw 16\nSkipping line 449079: expected 12 fields, saw 16\nSkipping line 450250: expected 12 fields, saw 15\n'
b'Skipping line 481374: expected 12 fields, saw 18\nSkipping line 510619: expected 12 fields, saw 20\nSkipping line 512271: expected 12 fields, saw 14\nSkipping line 514317: expected 12 fields, saw 15\nSkipping line 519707: expected 12 fields, saw 40\n'
b'Skipping line 538495: expected 12 fields, saw 13\n'
b'Skipping line 765901: expected 12 fields, saw 56\nSkipping line 773310: expected 12 fields, saw 43\n'
b'Skipping line 805899: expected 12 fields, saw 13\nSkipping line 868492: expected 12 fields, saw 21\nSkipping line 871801: expecte

In [0]:
df = df.drop(["url", "video_url", "source_name", "author_name"], axis=1)

In [0]:
df = df.dropna()
df = df.drop_duplicates()

In [0]:
df.head(5)

In [0]:
df['result.tsv'][12]

In [0]:
def normalize(text):
    
    normalized_text = [(word.strip(punctuation)) for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]

In [0]:
#def preprocess(text):
    text = nltk.word_tokenize(text)
    text = nltk.pos_tag(text)
    return text

In [0]:
#text = preprocess('result.tar.gz')
text

[('result.tar.gz', 'NN')]

In [0]:
def titles(fileid=None,corpus='result.tar.gz'):
    """
    Use a regular expression to extract the titles from the corpus.
    """
    pattern = re.compile(r'^(.*)[\s]+[\s]?(.*)?') 

    if fileid is not None:
        match = pattern.search('result.tar.gz'.raw(fileid))
        yield " ".join(map(lambda s: s.strip(), match.groups()))
    else:
        for fileid in corpus.fileids():
            # Search for a pattern match
            match = pattern.search(corpus.raw(fileid))

            if match:
                # If we find one, yield the space joined groups.
                yield " ".join(map(lambda s: s.strip(), match.groups()))
                
for idx, title in enumerate(titles()):
  print title
  if idx >= 10:
    break

SyntaxError: ignored

In [0]:
def sectpull(fileids=None, section=None, corpus='result.tar.gz'):
    """
    Uses a regular expression to pull sections from a file:
        - "top": everything until the references section
        - "ref": the references and anything that follows.
    Yields the text as top, ref respectively.
    """

    # Select either a single fileid or a list of fileids
    fileids = fileids or corpus.fileids()
    if section == None:
        section = None
    elif section == 'references':
        section = re.compile('(?<=' + 'REFERENCES' + ')(.+)')
    elif section == 'body':
        section = re.compile("(.+?)(?="+'REFERENCES'+ ")")
    elif section == 'top':
        section = ["url", "video_url", "source_name", "author_name"]
        for sect in section:
            try:
                section = re.compile("(.+?)(?="+sect+ ")")
                break
            except:
                pass

    # Iterate through all text for each file id.
    for fileid in fileids:
        text   = corpus.raw(fileid)

    # Extract the text and search for the section target
        if section == None:
            target = re.sub('[\s]', " ", text)
        else:
            text   = re.sub('[\s]', " ", text)
            target = section.search(text)
        if target:
            yield fileid, target.group(0), target.group(1)


def refstats(fileids=None, section=None, corpus='result.tar.gz'):
    """
    Code to pull only the references section, store a character count, number
    of references, as well as a "words per reference" count.
    Pass either a specific document id, a list of ids, or None for all ids.
    """

    # Create reference number to match pattern
    refnum  = re.compile(r'\[[0-9]{1,3}\]', re.I)

    for fileid, top, refs in sectpull(fileids, section, corpus):
      # Yield the statistics about the references
        n_refs = len(set((refnum.findall(refs))))
        
        words  = sum(1 for word in nltk.word_tokenize(refs))
        wp_ref = float(words) / float(n_refs) if n_refs else 0
        
        # Yield the data from the generator
        yield (fileid, len(refs), n_refs, wp_ref)

In [0]:
From tabulate import tabulate
from operator import itemgetter
import random

# Create table sorted by number of references
table   = 
sorted(list(refstats(random.sample(kddcorpus.fileids(),15),section='body')))

# Print the table with headers
headers = ('File', 'Characters', 'References', 'Words per Reference')

print(tabulate(table, headers=headers)

SyntaxError: ignored